# 檢索器策略比較

本範例展示：
1. **第1個儲存格**：載入向量資料庫
2. **第2個儲存格**：比較三種檢索策略（Similarity、MMR、Threshold）

學習目標：理解不同檢索策略的特點，根據需求選擇最適合的檢索方式

In [ ]:
# 第1個儲存格：載入向量資料庫

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

print("向量資料庫已載入")
print(f"資料庫位置: {persistent_directory}")

In [ ]:
# 第2個儲存格：比較三種檢索策略

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 載入向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 使用不同搜尋類型和參數查詢向量存儲的函數
def query_vector_store(search_type, search_kwargs, description):
    print(f"\n{'='*70}")
    print(f"檢索策略：{description}")
    print(f"{'='*70}")
    print(f"搜尋類型: {search_type}")
    print(f"搜尋參數: {search_kwargs}")
    print(f"{'-'*70}")
    
    retriever = db.as_retriever(
        search_type=search_type,
        search_kwargs=search_kwargs,
    )
    relevant_docs = retriever.invoke(query)
    
    # 顯示檢索結果數量
    print(f"\n找到 {len(relevant_docs)} 個相關文件\n")
    
    # 顯示相關結果
    for i, doc in enumerate(relevant_docs, 1):
        print(f"--- 文件 {i} ---")
        # 只顯示前 150 字元
        content_preview = doc.page_content[:150].replace('\n', ' ')
        print(f"{content_preview}...")
        if doc.metadata:
            print(f"來源: {doc.metadata.get('source', 'Unknown')}")
        print()

# 定義使用者的問題
query = "賈寶玉和林黛玉是什麼關係?"

print("\n" + "="*70)
print(f"查詢問題：{query}")
print("="*70)

# 展示不同的檢索方法

# 1. 相似度搜尋（Similarity Search）
# 說明：
# - 根據向量相似度檢索文件
# - 使用餘弦相似度找到與查詢向量最相似的文件
# - k: 返回最相似的前 k 個文件
# 適用場景：
# - 想要檢索最相關的文件
# - 不在意結果之間是否有重複內容
query_vector_store(
    "similarity",
    {"k": 3},
    "相似度搜尋 (Similarity Search)"
)

# 2. 最大邊際相關性（MMR - Maximal Marginal Relevance）
# 說明：
# - 在相關性和多樣性之間取得平衡
# - fetch_k: 初始獲取的文件數量（基於相似度）
# - lambda_mult: 控制多樣性（1=最小多樣性，0=最大多樣性）
# - k: 最終返回的文件數量
# 適用場景：
# - 想要避免重複或相似的結果
# - 需要更廣泛的資訊範圍
# - 希望結果涵蓋不同角度或觀點
query_vector_store(
    "mmr",
    {"k": 3, "fetch_k": 20, "lambda_mult": 0.5},
    "最大邊際相關性 (MMR)"
)

# 3. 相似度分數閾值（Similarity Score Threshold）
# 說明：
# - 只返回超過特定相似度分數的文件
# - score_threshold: 最低相似度分數（0-1之間）
# - k: 最多返回的文件數量
# 適用場景：
# - 只想要高度相關的結果
# - 寧願少返回也不要不相關的結果
# - 需要保證結果品質
query_vector_store(
    "similarity_score_threshold",
    {"k": 3, "score_threshold": 0.1},
    "相似度分數閾值 (Similarity Score Threshold)"
)

print("\n" + "="*70)
print("總結比較")
print("="*70)
print("\n1️⃣  相似度搜尋 (Similarity)")
print("   ✅ 優點: 返回最相關的結果，速度快")
print("   ❌ 缺點: 可能返回內容相似的重複結果")
print("   📌 適用: 一般查詢、快速檢索")

print("\n2️⃣  最大邊際相關性 (MMR)")
print("   ✅ 優點: 結果多樣化，涵蓋不同角度")
print("   ❌ 缺點: 可能犧牲部分相關性，速度較慢")
print("   📌 適用: 需要多樣化答案、探索性查詢")

print("\n3️⃣  相似度分數閾值 (Threshold)")
print("   ✅ 優點: 保證結果品質，過濾低相關性")
print("   ❌ 缺點: 可能返回較少結果或無結果")
print("   📌 適用: 品質優先、精確查詢")

print("\n" + "="*70)
print("建議：")
print("- 一般用途：使用 Similarity")
print("- 避免重複：使用 MMR (lambda_mult=0.5-0.7)")
print("- 高品質結果：使用 Threshold (score_threshold=0.7-0.9)")
print("="*70)